In [1]:
# import PyPDF2
import pdfplumber
import pikepdf
import os
import re

# Specify the path to the e-Aadhaar PDF file
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
pdf_path = os.path.join(downloads_folder, 'EAadhaar.pdf')  # Change to your actual file name

# Function to unlock the PDF
def unlock_pdf(input_pdf_path, password):
    output_pdf_path = input_pdf_path.replace('.pdf', '_unlocked.pdf')
    try:
        with pikepdf.open(input_pdf_path, password=password) as pdf:
            pdf.save(output_pdf_path) 
        return output_pdf_path
    except Exception as e:
        print(f"Error unlocking PDF: {e}")
        return None

# Function to filter English text
def filter_english_text(text):
    # Use regex to keep only English letters, numbers, spaces, and basic punctuation
    return re.sub(r'[^a-zA-Z0-9\s.,;:!?\'"()-]', '', text)

# Function to extract text using PyPDF2
def extract_text_with_pypdf2(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                page_text = page.extract_text()
                if page_text:  # Check if text was extracted
                    filtered_text = filter_english_text(page_text)
                    text += f"Page {page_num + 1}:\n{filtered_text}\n"
    except Exception as e:
        print(f"Error reading PDF with PyPDF2: {e}")
    return text

# Function to extract text using pdfplumber
def extract_text_with_pdfplumber(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num in range(len(pdf.pages)):
                page = pdf.pages[page_num]
                page_text = page.extract_text()
                if page_text:  # Check if text was extracted
                    filtered_text = filter_english_text(page_text)
                    text += f"Page {page_num + 1}:\n{filtered_text}\n"
    except Exception as e:
        print(f"Error reading PDF with pdfplumber: {e}")
    return text

# Specify your PDF password here
pdf_password = "CHAV2005"  # Change this to your actual password

# Unlock the PDF
unlocked_pdf_path = unlock_pdf(pdf_path, pdf_password)

# If unlocking was successful, extract text
if unlocked_pdf_path:
    print("Extracting text using PyPDF2 from unlocked PDF:")
    pypdf2_text = extract_text_with_pypdf2(unlocked_pdf_path)
    print(pypdf2_text)

    #print("Extracting text using pdfplumber from unlocked PDF:")
    #pdfplumber_text = extract_text_with_pdfplumber(unlocked_pdf_path)
    #print(pdfplumber_text)

Extracting text using PyPDF2 from unlocked PDF:
Error reading PDF with PyPDF2: name 'PyPDF2' is not defined



In [4]:
import PyPDF2
import pdfplumber
import pikepdf
import os
import re
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

# Specify the path to the e-Aadhaar PDF file
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
pdf_path = os.path.join(downloads_folder, 'EAadhaar.pdf')  # Change to your actual file name

# Load the Excel file directly from Documents folder (fake_data(1).xlsx)
documents_folder = os.path.join(os.path.expanduser('~'), 'DOwnloads')
excel_path = os.path.join(downloads_folder, 'fake_data(1).xlsx')  # Path to your Excel file
excel_data = pd.read_excel(excel_path)

# Function to unlock the PDF
def unlock_pdf(input_pdf_path, password):
    output_pdf_path = input_pdf_path.replace('.pdf', '_unlocked.pdf')
    try:
        with pikepdf.Pdf.open(input_pdf_path, password=password) as pdf:
            pdf.save(output_pdf_path)
        return output_pdf_path
    except Exception as e:
        print(f"Error unlocking PDF: {e}")
        return None

# Function to filter English text
def filter_english_text(text):
    return re.sub(r'[^a-zA-Z0-9\s.,;:!?\'"()-]', '', text)

# Function to extract text using PyPDF2
def extract_text_with_pypdf2(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                page_text = page.extract_text()
                if page_text:
                    filtered_text = filter_english_text(page_text)
                    text += f"Page {page_num + 1}:\n{filtered_text}\n"
    except Exception as e:
        print(f"Error reading PDF with PyPDF2: {e}")
    return text

class AadhaarVerificationApp(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self.title("Aadhaar Card Verification")
        self.geometry("600x400")

        # Create UI components
        self.create_widgets()

    def create_widgets(self):
        # Password Label and Entry
        tk.Label(self, text="Enter PDF Password:").pack(pady=10)
        self.password_entry = tk.Entry(self, show='*', width=30)
        self.password_entry.pack(pady=5)

        # Upload Button for PDF
        self.upload_button_pdf = tk.Button(self, text="Upload Aadhaar PDF", command=self.upload_pdf)
        self.upload_button_pdf.pack(pady=20)

        # Extracted Text Display Area
        self.text_area = scrolledtext.ScrolledText(self, wrap=tk.WORD, width=70, height=15)
        self.text_area.pack(pady=10)

    def upload_pdf(self):
        # Open file dialog to select PDF file
        pdf_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        
        if not pdf_path:
            return  # If no file is selected

        password = self.password_entry.get()
        
        # Unlock the PDF and extract text
        unlocked_pdf_path = unlock_pdf(pdf_path, password)

        if unlocked_pdf_path:
            print("Extracting text using PyPDF2 from unlocked PDF:")
            extracted_text = extract_text_with_pypdf2(unlocked_pdf_path)
            self.text_area.delete(1.0, tk.END)  # Clear previous content
            self.text_area.insert(tk.END, "Extracted Text:\n")
            self.text_area.insert(tk.END, extracted_text + "\n")
            
            # Validate Aadhaar card against loaded Excel data
            validation_result = self.validate_aadhaar(extracted_text)
            v = "Data Extraction is done !";
            messagebox.showinfo("Validation Result", v)

    def validate_aadhaar(self, extracted_text):
        # Extract relevant fields from the extracted text (example regex patterns)
        
        name_pattern = r'Name:\s*(.*)'  # Adjust based on actual text format in your PDF
        aadhaar_pattern = r'\b\d{4}\s\d{4}\s\d{4}\b'  # Example pattern for Aadhaar number
        
        name_match = re.search(name_pattern, extracted_text)
        aadhaar_match = re.search(aadhaar_pattern, extracted_text)

        if name_match and aadhaar_match:
            name = name_match.group(1).strip()
            aadhaar_number = aadhaar_match.group(0).replace(" ", "")  # Remove spaces
            
            # Check against Excel data
            matched_row = excel_data[
                (excel_data['Aadhaar Number'] == aadhaar_number) & 
                (excel_data['Name'].str.lower() == name.lower())
            ]

            if not matched_row.empty:
                return "Aadhaar card is valid."
            else:
                return "Aadhaar card is invalid."
        
        else:
            "Could not extract necessary information from the PDF."

if __name__ == "__main__":
    app = AadhaarVerificationApp()
    app.mainloop()


Extracting text using PyPDF2 from unlocked PDF:
